In [1]:
from config_parsing import parse_config
import plotly.io as pio

%load_ext autoreload
%autoreload 2

pio.renderers.default = "jupyterlab"
 
CONFIG_LOCATION = "D:/SteamLibrary/steamapps/common/Satisfactory/CommunityResources/Docs/Docs.json"
config = parse_config(CONFIG_LOCATION)

In [2]:
from material import MaterialSpec
from plotting.tables import recipe_summary
from plotting.sankey import plot_sankey
from plotting.graph import plot_process
from process import Process
from machine import Machine
from IPython.display import display, HTML
from igraph import Graph

def factory_plan(available_machines: list[Machine], target: MaterialSpec):
    Machine.autobind(available_machines)

    solution = Process.from_outputs(target,
                                    available_machines
                                    )

    # TODO: color by machine or recipe type
    plot_process(solution, Graph.layout_reingold_tilford)
    plot_sankey(solution)
    display(HTML(recipe_summary(solution)))
    # total_power(solution)
    return solution

In [32]:
# total production
config = parse_config(CONFIG_LOCATION)
available_machines = [m.instance() for m in config.recipes.values()]
target_product = config.materials(iron_rod=10, iron_plate=10, reinforced_iron_plate=5, rotor=5,
                                  steel_beam=5, encased_industrial_beam=2, modular_frame=5, heavy_modular_frame=5)

# factory_plan(available_machines, target_product)

In [3]:
available_machines = make_recipes(BASE_CONFIG, machine_mapping)
available_machines = [*available_machines.core]
Machine.autobind(available_machines)

target_product = MaterialSpec(iron_plate=1, iron_rod=2)
target_available = MaterialSpec(iron_ore=360)
solution = Process.from_inputs(target_available,
                               target_product,
                               available_machines)

plot_process(solution, Graph.layout_reingold_tilford)
plot_sankey(solution)
display(HTML(recipe_summary(solution)))

NameError: name 'make_recipes' is not defined

In [4]:
from plotting.widgets import material_widget, recipe_widget, optimize_outputs_widget, optimize_inputs_widget
from ipywidgets.widgets import Accordion
from IPython.display import display

a = None

def callback(process):
    global a
    a = process
    

m_w, materials = material_widget(config.materials)
r_w, recipes = recipe_widget(config.recipes)
o_b = optimize_outputs_widget(callback, materials, recipes)

accordion = Accordion(children=[m_w, r_w, o_b])
accordion.set_title(0, 'Materials')
accordion.set_title(1, 'Recipes')
accordion.set_title(2, 'Run')
accordion

Accordion(children=(interactive(children=(IntSlider(value=0, description='uranium_waste', max=1000, min=-1000)…

In [9]:
from ipywidgets import widgets
from categorized_collection import CategorizedCollection
from fuzzywuzzy import process
from IPython.display import display
from traitlets import Bool

options = CategorizedCollection({"a": 1, "b": 2, "c": 3, "d": 4})
options.set_tag("a", "tag_1")
options.set_tag("b", "tag_1")
options.set_tag("b", "tag_2")
options.set_tag("c", "tag_3")
options.set_tag("d", "tag_4")

options = config.recipes

fuzz_threshold = .5

search = widgets.Text(
    placeholder='Search Recipes',
    disabled=False,
)

class CustomToggle(widgets.Button):
    value = Bool(False).tag(sync=True)
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.on_click(self.toggle)
    
    def toggle(self, *_, **__):
        self.value = not self.value
        self.button_style = "info" if self.value else ""

category_selectors = {tag: CustomToggle(description=tag) for tag in options.tags}

item_selectors = {name: widgets.Checkbox(value=False, description=name, indent=False) for name in options.keys()}

filtered = []


        
def unselect_category_selectors(change):
    new_value = change["new"]
    if not new_value:
        for tag in options.value_tags(change["owner"].description):
            if (selector := category_selectors[tag]).value:
                selector.toggle()
            
    # TODO: if all in category selected, select category
            
for selector in item_selectors.values():
    selector.observe(unselect_category_selectors, names="value")
        
def select_by_category(b):
    tag = b.description
    # TODO: de-selecting a sub group de-selects a super group
    for item in options.tags[tag]:
        item_selectors[item].value = b.value

for selector in category_selectors.values():
    selector.on_click(select_by_category)

checkboxes = widgets.VBox(list(item_selectors.values())) 

def filter_results(change):
    new_value = change["new"]
    threshold = 75
    
    if not new_value:
        # TODO: cache this value
        filtered = list(item_selectors.values())
    else:
        filtered = [item_selectors[k] for k, score in process.extract(change["new"], options.keys(), limit=None) if score > threshold]
        
    checkboxes.children = filtered
    
def clear_search(b):
    search.value = ""

clear = widgets.Button(description="clear")
clear.on_click(clear_search)
    
search.observe(filter_results, names="value")
display(widgets.HBox([search, clear])) # TODO: clear button
display(widgets.HBox(list(category_selectors.values())))
display(checkboxes)